In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt


In [ ]:
df = pd.read_csv('../input/best-shopping/sale.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.dropna(inplace=True)

In [ ]:
def delet(x):
    if 'Quantity Ordered' in x :
         return np.nan
    else:
        return x

In [ ]:
df['Quantity Ordered'] = df['Quantity Ordered'].apply(delet)

In [ ]:
df['Quantity Ordered'].isnull().value_counts()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df['total'] = df['Quantity Ordered'].astype('int') * df['Price Each'].astype('float')

In [ ]:
df.drop(['Quantity Ordered', 'Price Each'], inplace=True, axis=1)

In [ ]:
def sep_addres(address):
    city = address.split(',')[1]
    pre_city = address.split(',')[2][:4]
    return city + pre_city
    

In [ ]:
df['Purchase Address'] = df['Purchase Address'].apply(sep_addres)

In [ ]:
df['Order Date'] = pd.to_datetime(df['Order Date'])

In [ ]:
def sep_month(time):
    return time.strftime('%m')

In [ ]:
df['month'] = df['Order Date'].apply(sep_month)

In [ ]:
def sep_year(time):
    return time.strftime('%Y')

In [ ]:
df['year'] = df['Order Date'].apply(sep_year)

In [ ]:
def sep_time(time):
    return time.strftime('%H')

In [ ]:
df['hour'] = df['Order Date'].apply(sep_time)

In [ ]:
df.drop('Order Date', axis=1, inplace=True)

In [ ]:
df

## Visualize:

In [ ]:
import seaborn as sns

In [ ]:
df['total'].value_counts()

In [ ]:
df['hour'].value_counts()

In [ ]:
g_h_t = df.pivot_table(index = 'hour', values='total', aggfunc='count')

In [ ]:
g_h_t

In [ ]:
g_h_t.plot(kind='bar', ylabel='count')

In [ ]:
g_m_t = df.pivot_table(index = 'month', values='total', aggfunc='count')

In [ ]:
g_m_t.sort_values(by='total', ascending=False)

In [ ]:
df['month'].value_counts()

In [ ]:
df['Purchase Address'].unique()

In [ ]:
df['Product'].unique()

In [ ]:
h_t_p = df.pivot_table(index = 'hour', values='total',columns='Product' ,aggfunc='count')

In [ ]:
h_t_p

In [ ]:
df

In [ ]:
df['Order ID'].is_unique

In [ ]:
df['order_product'] = df.groupby(['Order ID'])['Product'].transform(lambda x: ','.join(x))

In [ ]:
df1 = df[['Order ID', 'order_product']].drop_duplicates()

* with combination , combine 3 time then count the number of repetated time

* Counter count most repetative


In [ ]:
from itertools import combinations
from collections import Counter
count = Counter()
for row in df1['order_product']:
    #print(row)
    row_list = row.split(',')
    #print(row_list)
    count.update(Counter(combinations(row_list, 3)))
for key, value in count.most_common(10):
    print(key, value)

In [ ]:
df['Product'].values == 'iphone'

In [ ]:
def most_iphone(x):
        if 'iPhone' in x :
            return x
        else:
            return np.nan

In [ ]:
df['iphone'] = df['Product'].apply(most_iphone)

In [ ]:
df['iphone'].isnull().value_counts()

In [ ]:
df_iphone = df.drop(columns=['order_product', 'Product'])

In [ ]:
df_iphone = df_iphone.dropna()

In [ ]:
df_iphone

In [ ]:
df_iphone['Purchase Address'].value_counts()

In [ ]:
most_iphone_sale = df_iphone.pivot_table(index='Purchase Address', columns='hour',values='total',aggfunc='count')

In [ ]:
most_iphone_sale

In [ ]:
most_iphone_sale.max(axis='columns')

In [ ]:
most_iphone_sale.max(axis= 'columns').plot(kind='bar', figsize=(20,10))

In [ ]:
df.drop(columns=['iphone', 'order_product'], inplace=True)

# How Many order_id It's Not Have 1 Purchase:
# And Which Product Are More Use Fore Second Purchase 

In [ ]:
df['Order ID'].unique()
#df['Order ID'].shape

In [ ]:
df_multi_product = df
df_multi_product['order_id'] = df['Order ID'].duplicated()

In [ ]:
df_multi_product['order_id'] = df_multi_product['order_id'].astype('object')

In [ ]:
def return_duplicate(x):
    if x:
        return 1
    else:
        return np.nan

In [ ]:
df_multi_product['order_id'] = df_multi_product['order_id'].apply(return_duplicate)

In [ ]:
df_multi_product = df_multi_product.dropna()

In [ ]:
df_multi_product.drop('order_id',axis='columns' ,inplace=True)

In [ ]:
most_product_sale = df_multi_product.pivot_table(index='Product', columns='Purchase Address', values='total', aggfunc='count')

# You Can See Below The USB_C ,LIGHTING_CHARGING And Wired_Headphone Are Most Common Product Pepople Buy Together With Other Product

In [ ]:
most_product_sale.max(axis='columns').plot(kind='bar')

In [ ]:
df['hour'].value_counts()

In [ ]:
total_19 = df[ df['hour'] == '19' ] ['total']
total_18 = df[ df['hour'] == '13' ] ['total']

In [ ]:
total_19.mean() - total_18.mean()

In [ ]:
np.append(np.zeros(len(total_19)), np.ones(len(total_18)))

In [ ]:
sales = np.array([
    np.append(np.zeros(len(total_19)), np.ones(len(total_18))),
    np.append(total_18, total_19)
])

In [ ]:
sale_t = sales.T
np.random.shuffle(sale_t)

In [ ]:
ex = np.random.randint(0, 2, len(sale_t))

In [ ]:
ex_data = np.array([
    ex, 
    sale_t[:, 1]
])

In [ ]:
ex_data = ex_data.T

In [ ]:
print(ex_data[ex_data[:, 0] == 0 ][: ,1].mean())
print(ex_data[ex_data[:, 0] == 1 ][: ,1].mean())


In [ ]:
def sh_exp(n):
    ex_dif_mean = np.empty([n, 1])
    for time in np.arange(n):
        ex = np.random.randint(0, 2, len(sale_t))
        ex_data = np.array([
        ex, 
        sale_t[:, 1]]).T
        ex_dif_mean[time] = ex_data[ex_data[:, 0] == 1 ][: ,1].mean() - ex_data[ex_data[:, 0] == 0 ][: ,1].mean()
    return ex_dif_mean


        

In [ ]:
e = sh_exp(1000)

In [ ]:
len(e[e >= 9.271544781382346])

In [ ]:
17 / 1000 

b ehtemal %1 mobtani bar shanse bode
va 99% bar asas manadari bode


p_value = 0.1 
so reject h0 


In [ ]:
import seaborn as sns
sns.distplot(e, bins=100)